In [7]:
import wget
import numpy as np
import pandas as pd
from tqdm import tqdm

from abc import *
from pathlib import Path
import os
import tempfile
import shutil
import pickle

In [8]:
tqdm.pandas()

C:\Users\ngtze\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
data = pd.read_excel('../data/processed_data/full_data.xlsx')
data.drop(co)
data.head()

,Unnamed: 0,userId,movieId,rating,timestamp_ratings,tag,timestamp_tags,title,genres_x,MovieYear,...,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_y,averageRating,numVotes,directors,writers
0,0,4,7569,3.5,1573943431,so bad it's good,1573943455,You Only Live Twice,Action|Adventure|Sci-Fi|Thriller,1977,...,You Only Live Twice,0,1967,\N,117,"Action,Adventure,Thriller",6.9,99691,nm0318150,"nm0089169,nm0001094,nm0001220,nm0420845"
1,1,18853,7569,3.0,1248068143,007,1248068150,You Only Live Twice,Action|Adventure|Sci-Fi|Thriller,1977,...,You Only Live Twice,0,1967,\N,117,"Action,Adventure,Thriller",6.9,99691,nm0318150,"nm0089169,nm0001094,nm0001220,nm0420845"
2,2,18853,7569,3.0,1248068143,james bond,1248068148,You Only Live Twice,Action|Adventure|Sci-Fi|Thriller,1977,...,You Only Live Twice,0,1967,\N,117,"Action,Adventure,Thriller",6.9,99691,nm0318150,"nm0089169,nm0001094,nm0001220,nm0420845"
3,3,21096,7569,4.0,1269243364,franchise,1246471298,You Only Live Twice,Action|Adventure|Sci-Fi|Thriller,1977,...,You Only Live Twice,0,1967,\N,117,"Action,Adventure,Thriller",6.9,99691,nm0318150,"nm0089169,nm0001094,nm0001220,nm0420845"
4,4,21096,7569,4.0,1269243364,James Bond,1246471305,You Only Live Twice,Action|Adventure|Sci-Fi|Thriller,1977,...,You Only Live Twice,0,1967,\N,117,"Action,Adventure,Thriller",6.9,99691,nm0318150,"nm0089169,nm0001094,nm0001220,nm0420845"


In [ ]:
## For this model we only need the history of ratings
def getRatings()


## Get Size of the Data
def get_count(tp, id):
    groups = tp[[id]].groupby(id, as_index=False)
    count = groups.size()
    return count

#Get movies and users which share at least 3 intersection
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users.
    if min_sc > 0:
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]

    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]

    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId')
    return tp, usercount, itemcount

#Possible use to reduce size of data (we are only interested in the movies that were rated 4-5 )
def make_implicit(df, min_rating):
    print('Turning into implicit ratings')
    df = df[df['rating'] >= min_rating]
    # return df[['uid', 'sid', 'timestamp']]
    return df


#Probability Mass Distibution of user and movie
def densify_index(df):
    print('Densifying index')
    umap = {u: i for i, u in enumerate(set(df['uid']))}
    smap = {s: i for i, s in enumerate(set(df['sid']))}
    df['uid'] = df['uid'].map(umap)
    df['sid'] = df['sid'].map(smap)
    return df, umap, smap



In [ ]:
import pandas as pd

from datetime import date

#Create a Dataset Class that cr
class ML20MDataset(metaclass=ABCMeta):
    def __init__(self, args):
        self.args = args
        self.min_rating = args.min_rating
        self.min_uc = args.min_uc
        self.min_sc = args.min_sc
        self.split = args.split

        assert self.min_uc >= 2, 'Need at least 2 ratings per user for validation and test'
    
    def make_implicit(self, df):
        print('Turning into implicit ratings')
        df = df[df['rating'] >= self.min_rating]
        # return df[['uid', 'sid', 'timestamp']]
        return df



    def densify_index(self, df):
        print('Densifying index')
        umap = {u: i for i, u in enumerate(set(df['uid']))}
        smap = {s: i for i, s in enumerate(set(df['sid']))}
        df['uid'] = df['uid'].map(umap)
        df['sid'] = df['sid'].map(smap)
        return df, umap, smap

    def split_df(self, df, user_count):
        if self.args.split == 'leave_one_out':
            print('Splitting')
            user_group = df.groupby('uid')
            user2items = user_group.progress_apply(lambda d: list(d.sort_values(by='timestamp')['sid']))
            train, val, test = {}, {}, {}
            for user in range(user_count):
                items = user2items[user]
                train[user], val[user], test[user] = items[:-2], items[-2:-1], items[-1:]
            return train, val, test
        elif self.args.split == 'holdout':
            print('Splitting')
            np.random.seed(self.args.dataset_split_seed)
            eval_set_size = self.args.eval_set_size

            # Generate user indices
            permuted_index = np.random.permutation(user_count)
            train_user_index = permuted_index[                :-2*eval_set_size]
            val_user_index   = permuted_index[-2*eval_set_size:  -eval_set_size]
            test_user_index  = permuted_index[  -eval_set_size:                ]

            # Split DataFrames
            train_df = df.loc[df['uid'].isin(train_user_index)]
            val_df   = df.loc[df['uid'].isin(val_user_index)]
            test_df  = df.loc[df['uid'].isin(test_user_index)]

            # DataFrame to dict => {uid : list of sid's}
            train = dict(train_df.groupby('uid').progress_apply(lambda d: list(d['sid'])))
            val   = dict(val_df.groupby('uid').progress_apply(lambda d: list(d['sid'])))
            test  = dict(test_df.groupby('uid').progress_apply(lambda d: list(d['sid'])))
            return train, val, test
        else:
            raise NotImplementedError

    def all_raw_file_names(cls):
        return ['genome-scores.csv',
                'genome-tags.csv',
                'links.csv',
                'movies.csv',
                'ratings.csv',
                'README.txt',
                'tags.csv']

    
    def load_ratings_df(self):
        folder_path = self._get_rawdata_folder_path()
        file_path = folder_path.joinpath('ratings.csv')
        df = pd.read_csv(file_path)
        df.columns = ['uid', 'sid', 'rating', 'timestamp']
        return df

       def load_dataset(self):
        self.preprocess()
        dataset_path = self._get_preprocessed_dataset_path()
        dataset = pickle.load(dataset_path.open('rb'))
        return dataset

    def preprocess(self):
        dataset_path = self._get_preprocessed_dataset_path()
        if dataset_path.is_file():
            print('Already preprocessed. Skip preprocessing')
            return
        if not dataset_path.parent.is_dir():
            dataset_path.parent.mkdir(parents=True)
        self.maybe_download_raw_dataset()
        df = self.load_ratings_df()
        df = self.make_implicit(df)
        df = self.filter_triplets(df)
        df, umap, smap = self.densify_index(df)
        train, val, test = self.split_df(df, len(umap))
        dataset = {'train': train,
                   'val': val,
                   'test': test,
                   'umap': umap,
                   'smap': smap}
        with dataset_path.open('wb') as f:
            pickle.dump(dataset, f)


    def _get_rawdata_root_path(self):
        return Path(RAW_DATASET_ROOT_FOLDER)

    def _get_rawdata_folder_path(self):
        root = self._get_rawdata_root_path()
        return root.joinpath(self.raw_code())

    def _get_preprocessed_root_path(self):
        root = self._get_rawdata_root_path()
        return root.joinpath('preprocessed')

    def _get_preprocessed_folder_path(self):
        preprocessed_root = self._get_preprocessed_root_path()
        folder_name = '{}_min_rating{}-min_uc{}-min_sc{}-split{}' \
            .format(self.code(), self.min_rating, self.min_uc, self.min_sc, self.split)
        return preprocessed_root.joinpath(folder_name)

    def _get_preprocessed_dataset_path(self):
        folder = self._get_preprocessed_folder_path()
        return folder.joinpath('dataset.pkl')